In [1]:

import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
params[define._LAT_Y_] /= 2
params[define._LAT_XYZT_] /= 2
params[define._SET_PLAN_] = 2
params[define._DATA_TYPE_] = define._LAT_C64_
params[define._VERBOSE_] = 0
params[define._PARITY_] = 0
dtype = define.dtype(_data_type_=params[define._DATA_TYPE_])
gauge_filename = f"quda_wilson-clover-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
print("Parameters:", params)
#############################
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)
#############################
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
# qcu.applyWilsonDslashQcu(fermion_out, fermion_in, gauge, set_ptrs, params)
qcu.applyCloverDslashQcu(fermion_out, fermion_in, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))
#############################
fermion_in = cp.ones_like(fermion_in)
fermion_out = cp.zeros_like(fermion_in)
clover_ee = cp.zeros(shape=[define._LAT_S_, define._LAT_C_] +
                     list(fermion_in.shape), dtype=fermion_in.dtype)
clover_ee_inv = cp.zeros_like(clover_ee)
clover_oo = cp.zeros_like(clover_ee)
clover_oo_inv = cp.zeros_like(clover_ee)


@My Rank:0/1@

Parameters: [    32     16     32     32 524288      1      1      1      1      0
      0      1      0   1000      3      0      2      4      4      4
      8     24      0     42]
Gauge filename: quda_wilson-clover-dslash-gauge_-32-16-32-32-524288-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 16, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 16, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 16, 16)
Fermion in filename: quda_wilson-clover-dslash-fermion-in_-32-16-32-32-524288-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 16, Grid Lat X: 16
All Dest Shape: (4, 3, 32, 32, 16, 16)
Dest Shape: (4, 3, 32, 32, 16, 16)
Fermion out filename: quda_wilson-clover-dslash-fermion-out_-32-16-32-32-524288-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat 

In [9]:

#############################
params[define._VERBOSE_] = 0
params[define._PARITY_] = 0
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 2
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyCloversQcu(clover_ee, clover_ee_inv, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
params[define._VERBOSE_] = 0
params[define._PARITY_] = 1
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 2
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyCloversQcu(clover_oo, clover_oo_inv, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
params[define._VERBOSE_] = 0
params[define._PARITY_] = 0
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 1
#############################

lattice set whole time:0.171534622 sec
lattice set whole time:0.159508347 sec


In [19]:
index=1
print(clover_ee_inv[:,:,:,:,index,index,index,index].reshape(12,12))

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.

In [13]:

clover_ee = io.clover2I(input_array=clover_ee)
clover_oo = io.clover2I(input_array=clover_oo)
clover_ee_inv = io.clover2I(input_array=clover_ee_inv)
clover_oo_inv = io.clover2I(input_array=clover_oo_inv)
index=1
print(clover_ee_inv[:,:,:,:,index,index,index,index].reshape(12,12))


Input Array Shape: (4, 3, 4, 3, 32, 32, 16, 16)
Dest Shape: (12, 12, 262144)
Input Array Shape: (4, 3, 4, 3, 32, 32, 16, 16)
Dest Shape: (12, 12, 262144)
Input Array Shape: (4, 3, 4, 3, 32, 32, 16, 16)
Dest Shape: (12, 12, 262144)
Input Array Shape: (4, 3, 4, 3, 32, 32, 16, 16)
Dest Shape: (12, 12, 262144)
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0

In [20]:
A=clover_ee_inv[:,:,:,:,index,index,index,index].reshape(12,12)
x=cp.random.randn(4,3).view(dtype)
x=x.reshape(12)

In [21]:
A@x-x

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64)

In [22]:

#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyCloverBistabCgQcu(fermion_out, fermion_in,
                           gauge, clover_ee, clover_oo, clover_ee_inv, clover_oo_inv,  set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################


@@@@@@@@@@@@@@@DEBUGING!!!######TIME  :13.6493######
##RANK      :0
##LOOP      :-999
##tmp0      :(0,0i)
##tmp1      :(0,0i)
##rho_prev  :(0,0i)
##rho       :(0,0i)
##alpha     :(0,0i)
##beta      :(0,0i)
##omega     :(0,0i)
##send_tmp  :(3.14573e+06,0i)
##norm2_tmp :(3.14573e+06,0i)
##diff_tmp  :(0,0i)
##lat_4dim  :(262144,0i)
######TIME  :15.4489######
##RANK      :0
##LOOP      :-999
##tmp0      :(0,0i)
##tmp1      :(0,0i)
##rho_prev  :(0,0i)
##rho       :(0,0i)
##alpha     :(0,0i)
##beta      :(0,0i)
##omega     :(0,0i)
##send_tmp  :(4.71859e+06,0i)
##norm2_tmp :(4.71859e+06,0i)
##diff_tmp  :(0,0i)
##lat_4dim  :(262144,0i)
###############DEBUGING!!!##RANK:0##LOOP:999##Residual:(nan,nani)


KeyboardInterrupt: 